https://colab.research.google.com/drive/1jCkpikz0J2o20FBQmYmAGdiKmJGOMo-o?usp=sharing#scrollTo=T-gy-LxM0yAi

In [1]:
import os
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
checkpoint = "microsoft/biogpt"
from relations import relations
from datasets import DatasetDict, Dataset
import pandas as pd
from tqdm.notebook import trange, tqdm
from labels import get_labels

In [2]:
# load labels for bert_w_ner
additional_tokens, _, _, _ = get_labels(mode='GPT_w_ner')
print(additional_tokens, "\n", additional_tokens)

{'additional_special_tokens': ['[entity1]', '[entity2]', '[learn1]', '[learn2]', '[learn3]', '[learn4]', '[learn5]', '[learn6]', '[None]', '[Association]', '[Bind]', '[Comparison]', '[Conversion]', '[Cotreatment]', '[Drug_Interaction]', '[Negative_Correlation]', '[Positive_Correlation]']} 
 {'additional_special_tokens': ['[entity1]', '[entity2]', '[learn1]', '[learn2]', '[learn3]', '[learn4]', '[learn5]', '[learn6]', '[None]', '[Association]', '[Bind]', '[Comparison]', '[Conversion]', '[Cotreatment]', '[Drug_Interaction]', '[Negative_Correlation]', '[Positive_Correlation]']}


# load the model

In [3]:
# load the model in 8-bit quantization configuration
# the max length of the input is 1024
model = AutoModelForCausalLM.from_pretrained(checkpoint, 
    # load_in_8bit=True, 
    device_map={'':torch.cuda.current_device()},)

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# Tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")

In [6]:
# adding new tokens to the tokenizer
# since I haven't load the model so I will resize the embedding of the model later]
num_added_toks = tokenizer.add_special_tokens(additional_tokens)
print('We have added', num_added_toks, 'tokens')

# save the tokenizer
# tokenizer.save_pretrained("GPT_w_ner/GPT_w_ner_tokenizer")

We have added 17 tokens


In [7]:
model.resize_token_embeddings(len(tokenizer))

Embedding(42401, 1024)

# PEFT

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in float32 for stability.

We also cast the output of the last layer and embedding layer in float32 for the same reasons.

In [8]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)

model.biogpt.embed_tokens = CastOutputToFloat(model.biogpt.embed_tokens)
model.output_projection = CastOutputToFloat(model.output_projection)

In [9]:
# more with LoRAconfig: https://huggingface.co/docs/peft/conceptual_guides/lora

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, PeftType

peft_config = LoraConfig(
    # r: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters.
    r=16,
    # alpha: LoRA scaling factor.
    lora_alpha=32, 
    # target_modules: The modules (for example, attention blocks) to apply the LoRA update matrices.
    target_modules=["q_proj", "v_proj"],
    fan_in_fan_out=True,
    lora_dropout=0.05,
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
print_trainable_parameters(model)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/tian/mambaforge/envs/BioRED did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/lib/x86_64-linux-gnu/mesa'), PosixPath('/usr/lib/mesa-diverted/x86_64-linux-gnu'), PosixPath('/usr/lib/x86_64-linux-gnu/gallium-pipe')}
  warn(msg)
/home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib/mesa-diverted/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/mesa:/usr/lib/x86_64-linux-gnu/dri:/usr/lib/x86_64-linux-gnu/gallium-pipe did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further pat

trainable params: 1572864 || all params: 348353536 || trainable%: 0.45151371737475343


In [10]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BioGptForCausalLM(
      (biogpt): BioGptModel(
        (embed_tokens): CastOutputToFloat(
          (0): Embedding(42401, 1024)
        )
        (embed_positions): BioGptLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-23): 24 x BioGptDecoderLayer(
            (self_attn): BioGptAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(
                in_features=1024, out_features=1024, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
                )
                (lora_embedding_A): Pa

# pre-process the text

In [11]:
from data_preprocessing import make_GPT_re_data, GPT_w_ner_preprocess_function

In [12]:
# train and valid file paths
train_file_path = 'data/BioRED/processed/train.tsv'
valid_file_path = 'data/BioRED/processed/dev.tsv'

In [13]:
# make bert_re data
train_data_raw = make_GPT_re_data(file_path=train_file_path, lower=True)
valid_data_raw = make_GPT_re_data(file_path=valid_file_path, lower=True)

Dropped 10 line:
 [4548, 6646, 6758, 6776, 6866, 10222, 11775, 14657, 18818, 21689]
Dropped 9 line:
 [467, 941, 2220, 2233, 2261, 5335, 5337, 5378, 5490]


In [14]:
train_data_raw.keys()

dict_keys(['pmids', 'text', 'entities', 'outputs'])

In [15]:
# make into Dataset type
train_data_raw = Dataset.from_dict(train_data_raw)
valid_data_raw = Dataset.from_dict(valid_data_raw)

dataset = DatasetDict({
    "train": train_data_raw,
    "valid": valid_data_raw
})

In [16]:
tokenized_datasets = dataset.map(lambda example: GPT_w_ner_preprocess_function(example, tokenizer, mode="gpt_w_ner"), batched=True, remove_columns=['pmids', 'text', 'entities', 'outputs'])

Map:   0%|          | 0/22885 [00:00<?, ? examples/s]

Map:   0%|          | 0/6649 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 22885
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 6649
    })
})

In [18]:
# to tensor
tokenized_datasets.set_format(type='torch', columns=['input_ids'])

In [19]:
# from transformers import  default_data_collator
# from torch.utils.data import DataLoader


# # Set pin_memory=True to speed up the data transfer to the GPU during training if the samples in your dataset are on a CPU.
# # https://huggingface.co/docs/peft/task_guides/clm-prompt-tuning#train
# train_dataloader = DataLoader(
#     tokenized_datasets['train'], shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
# )

# eval_dataloader = DataLoader(tokenized_datasets['valid'], collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

# Training

wandb

In [20]:
import wandb

wandb.init(
    # set the wandb project where this run will be logged
    project="GPT2",
    # notes="PubmedBERT-FT-NER_w_NERin_10epochs",
    name="BioGPT_w_ner_epoch_30",
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 309439737 (tian1995). Use `wandb login --relogin` to force relogin


training

In [21]:
from transformers import DataCollatorForLanguageModeling

In [22]:
import transformers
from datasets import load_dataset

trainer = transformers.Trainer(
    model=model, 
    train_dataset=tokenized_datasets['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8, 
        gradient_accumulation_steps=8,
        warmup_steps=1000, 
        num_train_epochs=30,
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=1, 
        report_to="wandb",
        save_strategy="epoch",
        output_dir='GPT_w_ner'
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10710 [00:00<?, ?it/s]

{'loss': 3.1846, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.0}
{'loss': 3.1973, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.01}
{'loss': 3.265, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.01}
{'loss': 3.3118, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.01}
{'loss': 3.2424, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 3.246, 'learning_rate': 1.2000000000000002e-06, 'epoch': 0.02}
{'loss': 3.2188, 'learning_rate': 1.4000000000000001e-06, 'epoch': 0.02}
{'loss': 3.2384, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.02}
{'loss': 3.1838, 'learning_rate': 1.8e-06, 'epoch': 0.03}
{'loss': 3.247, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.03}
{'loss': 3.2912, 'learning_rate': 2.2e-06, 'epoch': 0.03}
{'loss': 3.2344, 'learning_rate': 2.4000000000000003e-06, 'epoch': 0.03}
{'loss': 3.2204, 'learning_rate': 2.6e-06, 'epoch': 0.04}
{'loss': 3.1984, 'learning_rate': 2.8000000000000003e-06, 'epoch': 0.04}
{'loss': 3.2764, 'learning_ra

TrainOutput(global_step=10710, training_loss=0.6543093271184813, metrics={'train_runtime': 161702.5355, 'train_samples_per_second': 4.246, 'train_steps_per_second': 0.066, 'train_loss': 0.6543093271184813, 'epoch': 29.95})

In [23]:
import wandb
wandb.finish()
trainer.save_model("GPT_w_ner/models/GPT_w_ner_epoch_30")

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▂▄▆▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▆▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,29.95
train/global_step,10710


In [24]:
model.save_pretrained("GPT_w_ner/models/GPT_w_ner_epoch_30_save_pretrained")

In [7]:
# Since there are key-unmatches in the trainer.save_model(), we need to rename the keys and load the paras in the model

embed_tokens_state_dict = torch.load("GPT_w_ner/models/GPT_w_ner_epoch_30/pytorch_model.bin")

old_keys = ["base_model.model.biogpt.embed_tokens.0.weight", "base_model.model.output_projection.0.weight"]
new_keys = ["base_model.model.biogpt.embed_tokens.weight", "base_model.model.output_projection.weight"]

for old_key, new_key in zip(old_keys, new_keys):
    # Get the value of the old key
    value = embed_tokens_state_dict[old_key]

    # Create a new key-value pair with the updated name
    embed_tokens_state_dict[new_key] = value

    # Delete the old key if desired
    del embed_tokens_state_dict[old_key]

torch.save(embed_tokens_state_dict, "GPT_w_ner/models/GPT_w_ner_epoch_30/pytorch_model-111.bin")

In [25]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BioGptForCausalLM(
      (biogpt): BioGptModel(
        (embed_tokens): CastOutputToFloat(
          (0): Embedding(42401, 1024)
        )
        (embed_positions): BioGptLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-23): 24 x BioGptDecoderLayer(
            (self_attn): BioGptAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(
                in_features=1024, out_features=1024, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
                )
                (lora_embedding_A): Pa

# load model and inference

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "microsoft/biogpt"

peft_model_id = "GPT_w_ner/models/GPT_w_ner_epoch_30_saved_peft"
# config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained("GPT_w_ner/GPT_w_ner_tokenizer")

# resize the token embeddings to match the tokenizer
model.resize_token_embeddings(len(tokenizer))

# Load the Lora model
# the resized embedding layer are still uncorrected, need to load the weights manually
model = PeftModel.from_pretrained(model, peft_model_id)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/tian/mambaforge/envs/BioRED did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/lib/x86_64-linux-gnu/mesa'), PosixPath('/usr/lib/mesa-diverted/x86_64-linux-gnu'), PosixPath('/usr/lib/x86_64-linux-gnu/gallium-pipe')}
  warn(msg)
/home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib/mesa-diverted/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/mesa:/usr/lib/x86_64-linux-gnu/dri:/usr/lib/x86_64-linux-gnu/gallium-pipe did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further pat

In [3]:
model.load_state_dict(torch.load("GPT_w_ner/models/GPT_w_ner_epoch_30/pytorch_model-111.bin"))

<All keys matched successfully>

In [2]:
model.load_state_dict(torch.load("GPT_w_ner/models/GPT_w_ner_epoch_10/pytorch_model.bin"))

<All keys matched successfully>

In [6]:
model.eval()
inputs = tokenizer("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label :", return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Tweet text: @ HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label: the word "smart" means that we are aware


In [1]:
import pandas as pd
import re
from tqdm.notebook import trange, tqdm
from torch import nn
from labels import get_labels
from relations import relations
from datasets import DatasetDict, Dataset

# from data_preprocessing import make_GPT_re_data, GPT_w_ner_preprocess_function
additional_tokens, _, _, _ = get_labels(mode='GPT_w_ner')

In [2]:
from data_preprocessing import all_line_of_pmid, get_original_text, get_identifier_and_entity, reorder_list

In [5]:
# load test data and preprocess
test_file_path = 'data/BioRED/processed/test.tsv'
test_data = make_GPT_re_data(file_path=test_file_path, lower=True)

test_dataset_raw = Dataset.from_dict(test_data)
# test_dataset = test_dataset_raw.map(NER_preprocess_function, batched=False)
# with bert only:
test_dataset = test_dataset_raw.map(lambda example: GPT_w_ner_preprocess_function(example, tokenizer, mode="gpt_w_ner", infer=True), batched=True, remove_columns=['pmids', 'text', 'entities', 'outputs'])
test_dataset.set_format(type='torch', columns=['input_ids'])
# the test_dataset has two columns: input_ids and labels, split the labels coloumn into test_dataset_labels

Dropped 2 line:
 [1001, 5977]


Map:   0%|          | 0/7588 [00:00<?, ? examples/s]

In [5]:
file_path = 'data/BioRED/processed/test.tsv'
lower=True

In [13]:
# def make_GPT_re_data(file_path, lower=True, no_ner_input=False):
"""make a dictionary for the dataset for GPT_re
input is the .tsv file path
return the hugging face dataset
data_dict = {
    "pmids": [],
    "text": [],
    "entities": [],
    "outputs": []
}

"""

data_dict = {
    "pmids": [],
    "text": [],
    "entities": [],
    "outputs": []
}
dropped = []
dataset = pd.read_csv(file_path, delimiter="\t", header=None)

# relations_dict = {relations[i]: f"[RELATION{i}]" for i in range(len(relations))}

# tag_dict = {}
start = 0
# while start < (len(dataset) - 1):
while start == 0:
    pmid, start, end = all_line_of_pmid(dataset, start)
    # the text
    text = get_original_text(dataset, start, end, lower)

    pmid = dataset.iloc[start, 0]

    # for i in range(start, start + 1):
    for i in range(start, end):
        entity_line = ""
        output_line = ""
        # get entities and their identifiers of this line. the 1st item of entities is the src and the 2nd is the tgt
        entities, entity_to_identifier = get_identifier_and_entity(dataset, i, i + 1, lower)

        # reorder the entities, find the first occurred entity as the entity1
        reordered_e1 = reorder_list(text, {list(entities.items())[0][0]:list(entities.items())[0][1]}, lower)
        reordered_e2 = reorder_list(text, {list(entities.items())[1][0]:list(entities.items())[1][1]}, lower)
        
        # get a string with the items in the reordered_e1, and split each item with ","
        entity1 = " ; ".join(reordered_e1)
        entity2 = " ; ".join(reordered_e2)
        # get the entity line
        entity_line = "[entity1] : " + entity1 + " . [entity2] : " + entity2 + " . "

        # get the output line
        # if none
        if dataset.iloc[i, 8] == "None":
            output_line = "the relation between source [entity1] and target [entity2] is [None] . "
        else:
            print("not None")
            try:
                if entity_to_identifier[reordered_e1[0]]['identifier'] == dataset.iloc[i, 3]:
                    # entity1 is the source
                    output_line = "the relation between source [entity1] and target [entity2] is [" + dataset.iloc[i, 8] + "] . "
                elif entity_to_identifier[reordered_e2[0]]['identifier'] == dataset.iloc[i, 3]:
                    # entity2 is the source
                    output_line = "the relation between source [entity2] and target [entity1] is [" + dataset.iloc[i, 8] + "] . "
            except:
                # if there is no such key of entity_to_identifier, drop this line
                dropped.append(i)
                continue
        data_dict["pmids"].append(pmid)
        data_dict["text"].append(text)
        data_dict["entities"].append(entity_line)
        data_dict["outputs"].append(output_line)

    start = end
print(f"Dropped {len(dropped)} line:\n {dropped}")
# print(data_dict)

not None
not None
not None
not None
not None
not None
not None
not None
not None
not None
not None
not None
not None
not None
not None
not None
not None
not None
Dropped 0 line:
 []


In [14]:
data_dict["outputs"]

['the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [entity2] is [Negative_Correlation] . ',
 'the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [entity2] is [Negative_Correlation] . ',
 'the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [entity2] is [None] . ',
 'the relation between source [entity1] and target [enti

In [18]:
test_dataset_raw['outputs'][5]

'the relation between source [entity1] and target [entity2] is [None] . '

In [7]:
model.eval()
outputs = []
model.to("cuda")
with torch.no_grad():
    for i in tqdm(range(len(test_dataset))):
    # for i in range(1):
        output = model.generate(input_ids=test_dataset[i]["input_ids"].unsqueeze(0).to("cuda"), max_new_tokens=50)
        output_text = tokenizer.batch_decode(output.detach().cpu().numpy(), skip_special_tokens=False)[0]
        outputs.append(output_text.split("[learn1] [learn2] [learn3] [learn4] [learn5] [learn6] ")[1])

    # print(tokenizer.batch_decode(output.detach().cpu().numpy(), skip_special_tokens=False)[0])

  0%|          | 0/7588 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
outputs

['the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [entity2] is [None]. </s>',
 'the relation between source [entity1] and target [

In [15]:
test_dataset['labels'][28]

''

In [10]:
for i in range(len(test_dataset)):
    if "[None]" not in test_dataset['labels'][i]:
        print(i)

6
8
15
28
33
38
40
44
49
56
60
61
68
72
86
94
99
103
114
116
117
122
126
129
132
134
135
138
140
141
145
146
147
148
154
157
174
179
186
188
189
190
192
197
198
203
205
209
213
215
219
222
227
234
235
236
239
243
244
247
251
254
260
267
269
271
272
275
277
279
280
285
289
292
293
298
299
300
304
310
311
315
317
318
319
323
324
327
328
333
336
340
353
354
356
367
372
374
380
382
384
387
391
397
406
408
409
412
416
424
427
436
438
446
450
452
454
456
457
460
461
463
533
568
632
641
678
705
777
821
836
855
864
870
876
886
908
911
912
925
931
939
947
948
951
952
961
965
970
974
985
987
988
990
993
998
1004
1009
1027
1035
1038
1046
1050
1054
1055
1064
1069
1071
1080
1086
1087
1089
1090
1092
1098
1103
1105
1108
1114
1116
1118
1119
1121
1123
1127
1132
1133
1136
1138
1143
1146
1147
1149
1150
1152
1153
1162
1182
1188
1189
1191
1197
1203
1208
1215
1225
1245
1274
1283
1288
1298
1330
1339
1364
1379
1381
1422
1424
1443
1465
1468
1472
1476
1489
1490
1493
1496
1501
1507
1508
1512
1530
1538
1540
1541


KeyboardInterrupt: 

In [ ]:
result = {
    "output": [],
    "label": []
}

for output, label in zip(outputs, test_dataset['labels']):
    result['output'].append(output)
    result['label'].append(label)

In [24]:
result

{'output': ['the relation between source [entity1] and target [entity2] is [None]. </s>',
  '</s>',
  '</s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  '</s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',
  'the relation between source [entity1] and target [entity2] is [None]. </s>',


In [ ]:
# save the result dictionary
import pickle
with open("GPT_w_ner/result/GPT_w_ner_epoch_30_result.pkl", "wb") as f:
    pickle.dump(result, f)

post-processing and evaluation

In [ ]:
# load the result dictionary
import pickle
with open("GPT_w_ner/result/GPT_w_ner_epoch_30_result.pkl", "rb") as f:
    result = pickle.load(f)